# 6. Generating square geotiffs

Now that we have selected a collection of representative squares, we can use them to filter the FeatureCollection asset we uploaded to Earth Engine earlier. With that, we will export geotiffs of all the trees that span a particular boundary.

In [1]:
from datetime import datetime
import ee
import geemap
import pandas as pd

In [4]:
def run():
    
    # Initializes the Earth Engine instance
    ee.Initialize(project='dw-city-tree-coverage')
    
    # A timestamp
    now = datetime.now().strftime("%d-%m-%y-%H:%M").replace(":","_")
    
    # Loads the tree coverage information from EE, performing the same steps as in the rest of the analysis
    tree_coverage_1m = ee.ImageCollection('projects/meta-forest-monitoring-okw37/assets/CanopyHeight').mosaic()
    tree_coverage_10m = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1')
    tree_coverage_1m = tree_coverage_1m.updateMask(tree_coverage_1m.gte(1))
    tree_coverage_10m = tree_coverage_10m.updateMask(tree_coverage_10m.gte(1))
    tree_coverage_mask = tree_coverage_10m.gte(1)
    tree_coverage = tree_coverage_1m.updateMask(tree_coverage_mask)
    
    # Loads the squares FeatureCollection
    squares = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/1sqkm_squares")
    
    # Reads the CSV with the squares we want to export images from.
    city_squares = pd.read_csv("../output/representative-squares-most-populated.csv")
    
    # Creates a single id for each square, combining city_id and hexagon_n
    city_squares['unique_id'] = city_squares.city_id.astype(str) + "_" + city_squares.hexagon_n.astype(str)
    
    # Does the same for the Feature Collection
    squares = squares.map(lambda feature: feature.set({
        "unique_id": ee.String(feature.get('city_id')).cat("_").cat(ee.String(feature.get('hexagon_n'))),
        
    }))
    
    # A function that will export all the squares listed in the CSV
    def export_images(df, type_):
        
        # For each row in the dataframe, select the desired polygon and export it
        for index, row in df.iterrows():

            # Gets information for the file name
            city_id = row.city_id
            hexagon_n = row.hexagon_n
            unique_id = row.unique_id

            # Makes up the file name. We will later split that string and extract the information
            # to join it back with the CSV files generated earlier.
            description = f"ImageSquare{type_}_{city_id}_{hexagon_n}_{now}"

            # Selects the relevant square geometry by filtering all the squares
            aoi = squares.filter(ee.Filter.eq('unique_id', unique_id)).first().geometry()

            #Exports
            geemap.ee_export_image_to_drive(
                tree_coverage, 
                description=description, 
                folder="DWTreeCoverage-2024-09-24", 
                region=aoi, 
                scale=1
            )

    export_images(city_squares, 'WholeCity')
   #export_images(bin_squares, 'BinLevels')

In [5]:
run()